In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [ ]:
df1 = pd.read_csv('../input/food-prices-in-turkey/train.csv')
df2 = pd.read_csv('../input/food-prices-in-turkey/test.csv')

In [ ]:
df1.shape, df2.shape

In [ ]:
df1.isna().sum()

In [ ]:
df2.isna().sum()

In [ ]:
df = pd.concat([df1, df2], axis=0)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(['ProductId', 'UmId'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['ProductName'].value_counts()

In [ ]:
cols_to_replace = df['ProductName'].value_counts()[df['ProductName'].value_counts() < 50].keys()

In [ ]:
df['ProductName'].replace(cols_to_replace,['Others' for i in range(6)],inplace=True)

In [ ]:
df['ProductName'].value_counts()['Others']

In [ ]:
plt.figure(figsize=(15,15))
df['ProductName'].value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df.dtypes

In [ ]:
df.Place.value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
df.UmName.value_counts()

In [ ]:
df['Place'].value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_dummies = pd.get_dummies(df[['Place', 'ProductName', 'UmName']],drop_first=True)

In [ ]:
df_dummies.columns

In [ ]:
df['Year'] = df['Year']/10000
df['Month'] = df['Month']/100


In [ ]:
df.head()

In [ ]:
df_main = pd.concat([df_dummies, df[['Month', 'Year', 'Price']]], axis=1)

In [ ]:
df_main.shape

In [ ]:
df_main.head()

In [ ]:
x, y = df_main.drop(['Price'],axis=1), df_main['Price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)

In [ ]:
model.score(x_test, y_test)

In [ ]:
model.score(x_train, y_train)

In [ ]:
mean_squared_error(y_test, y_pred_test)

In [ ]:
mean_squared_error(y_train, y_pred_train)

In [ ]:
[float(i) for i in cross_val_score(LinearRegression(), x, y, cv=10)]

In [ ]:
[float(i) for i in cross_val_score(Ridge(), x, y, cv=9)]

In [ ]:
x_train.to_csv('MyTrain.csv',index=False)
x_test.to_csv('MyTest.csv',index=False)

In [ ]:
import os
os.listdir()